# Universal noise

This notebook computes a universal noise pattern, which can be added to any image 
and missleads the open pose network 

In [1]:
%load_ext autoreload
%autoreload 2
# imports
import datetime
import logging
import os
import pathlib
import sys

import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import seaborn as sns
import copy


from torch_openpose.body import Body
from torch_openpose import util

from PIL import Image
import glob 
import wandb
import random
import torch
from torch.autograd import Variable
import torchvision.transforms as transforms

sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../..'))


logging.getLogger("wandb").setLevel(logging.ERROR)
random.seed(42)


Bad key "text.kerning_factor" on line 4 in
/home/ax/miniconda3/envs/masterThesisPytorchOpenpose/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/home/ax/.local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
def read_imgfile(path, width=None, height=None, image_type=cv2.IMREAD_COLOR):
    val_image = cv2.imread(path, image_type)
    if width is not None and height is not None:
        val_image = cv2.resize(val_image, (width, height))
    return val_image


def load_batch(training_paths, start_index, batch_size, width=432, height=368):
    i = start_index
    batch = []
    while len(batch) < batch_size:
        if i >= len(training_paths):
            i = 0
        batch.append(read_imgfile(str(training_paths[i]), width, height))
        i += 1
    return np.asarray(batch), i

def batch_to_input_tensor(batch, stride=8, padValue=128):
    tensor_list = []
    transform_func = transforms.ToTensor()
    for image in batch:
        imageToTest_padded, _ = util.padRightDownCorner(image, stride, padValue)
        tensor_list.append(
            transform_func(np.ascontiguousarray(imageToTest_padded))
        )

    return torch.stack(tensor_list)
def loss_function(batch_tensor, target_heat_tensor, target_paf_tensor):
    loss_fn = torch.nn.MSELoss(reduction='sum')
    
    batch_size = batch_tensor.shape[0]
    target_heat_tensor_batch = target_heat_tensor.repeat(batch_size, 1, 1, 1)
    target_paf_tensor_batch = target_paf_tensor.repeat(batch_size, 1, 1, 1)
    
    test_batch = Variable(batch_tensor, requires_grad=True)
    
    # compute heatmaps
    paf_tensor, heat_tensor = body_estimation.model(test_batch)

    # compute loss for heat and paf
    loss_heat = loss_fn(heat_tensor, target_heat_tensor_batch)
    loss_paf = loss_fn(paf_tensor, target_paf_tensor_batch)

    loss_total = loss_heat + loss_paf

    # get the total loss as a number
    total_loss_value = loss_total.item()

    # Zero the gradients before running the backward pass.
    body_estimation.model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss_total.backward(retain_graph=True)
    
    gradient = torch.mean(test_batch.grad, dim=0)
    return gradient, total_loss_value


In [3]:
# run config
IMAGE_WIDTH, IMAGE_HEIGHT = 432, 368
OUTPUT_WIDTH, OUTPUT_HEIGHT = 54, 46
image_path_target = '../data/images/dri_target_far_2.jpg'
image_path_source = '../data/images/dri_source_2.jpg'
train_dir = '../data/cocoval2017Humans/train'
test_dir = '../data/cocoval2017Humans/test'
image_target = read_imgfile(image_path_target, width=IMAGE_WIDTH, height=IMAGE_HEIGHT)
image_source = read_imgfile(image_path_source, width=IMAGE_WIDTH, height=IMAGE_HEIGHT)


run = wandb.init(
    project = "18_adv_universal_torch", 
    name = 'make it work',
    config={
        "epochs": 1,
        "epsilon": 5000,
        "batch_size": 16,
        "image_width": IMAGE_WIDTH,
        "image_height": IMAGE_HEIGHT,
    }
)

training_paths = pathlib.Path(train_dir).glob('*.jpg')
training_paths = sorted([x for x in training_paths])
test_paths = pathlib.Path(test_dir).glob('*.jpg')
test_paths = sorted([x for x in test_paths])
body_estimation = Body('/home/ax/Programs/pytorch-openpose/model/body_pose_model.pth')

In [4]:
# get the target heat and paf
input_tensor = body_estimation.image_to_input_tensor(image_target)
target_heat_np, target_heat_tensor = body_estimation.compute_heatmap(input_tensor)
target_paf_np, target_paf_tensor = body_estimation.compute_paf(input_tensor)
print(f"target_heat_np.shape= {target_heat_np.shape}")
print(f"target_paf_np.shape= {target_paf_np.shape}")

target_heat_np.shape= (46, 54, 19)
target_paf_np.shape= (46, 54, 38)


In [11]:
batch, _ = load_batch(training_paths, 0, 4)
print(f"batch.shape = {batch.shape}")
batch_tensor = batch_to_input_tensor(batch)
print(f"batch_tensor.shape = {batch_tensor.shape}  {batch_tensor.type()}")

grad, loss = loss_function(batch_tensor, target_heat_tensor, target_paf_tensor)
print(loss)
print(grad.shape)
print(body_estimation.image_to_input_tensor(image_source).shape)
print(loss)
print(grad.shape)

plt.imshow(grad_image / 256.0)

batch.shape = (4, 368, 432, 3)
batch_tensor.shape = torch.Size([4, 3, 368, 432])  torch.FloatTensor


KeyboardInterrupt: 

In [5]:
if not "training_stats" in vars():
    print("init")
    # where does the current batch start
    training_stats = {
        'gobal_step': 0,
        'batch_index': 0,
        'universal_noise': np.zeros((3, IMAGE_HEIGHT, IMAGE_WIDTH), dtype=np.float32),
        'file_name': 'universal_noise_torch'
    }
    
steps = len(training_paths) // run.config.batch_size
print(f"{steps} per epoch")

init
250 per epoch


In [6]:
### ================= TRAINING ================= ###
for epoch in range(run.config.epochs):
    print(f"run {epoch + 1}/{run.config.epochs}")
    for step in tqdm(range(steps)):        
        batch, training_stats['batch_index'] = load_batch(training_paths, training_stats['batch_index'], wandb.config.batch_size)

        repeated_noise = np.repeat(np.expand_dims(training_stats['universal_noise'], axis=0), wandb.config.batch_size, 0)
        
        
        
        batch_tensor = batch_to_input_tensor(batch)
        batch_tensor_noise = batch_tensor + torch.tensor(repeated_noise)

        gradient, loss_val = loss_function(batch_tensor_noise, target_heat_tensor, target_paf_tensor)

        wandb.log({'loss': loss_val}, step=training_stats['gobal_step'])
        

        ### ================= UPDATE STEP ================= ###
        scaled_gradient = (run.config.epsilon * gradient.numpy())            

        training_stats['universal_noise'] = training_stats['universal_noise'] - scaled_gradient
        training_stats['universal_noise'] = np.clip(training_stats['universal_noise'], 0, 255)
        training_stats['gobal_step'] += 1

  0%|          | 0/250 [00:00<?, ?it/s]

run 1/1


  1%|          | 3/250 [04:25<5:56:54, 86.70s/it]

KeyboardInterrupt: 